# Example 4: Working With Tuning

This notebook demonstrates how to work with hyperparameter spaces and configurations of tunable components.

## Set Up

As in previous examples, we will use the benchmarks package to get our system and task, and to generate a trajectory dataset.

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()

# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=10, traj_len=200)

Loading AutoMPC...
Finished loading AutoMPC


## Tunables

AutoMPC components -- models, optimizers, and optimal control problem transformers -- derive from the `Tunable` class to represent their hyperparameter configurations.

First, let's instantiate any such object to examine how it behaves.  Here we use the MLP (multi-layer perceptron) system ID model.

In [2]:
from autompc.sysid import MLP

model = MLP(system)

/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


A Tunable plays a double role as a factory for tunable objects as well as the objects themselves. A tunable object has a hyperparameter configuration space retrieved as follows:

In [3]:
print(model.get_config_space())

Configuration space object:
  Hyperparameters:
    batchnorm, Type: Categorical, Choices: {False, True}, Default: False
    hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_2, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_3, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_4, Type: UniformInteger, Range: [16, 256], Default: 128
    lr, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.001, on log-scale
    n_hidden_layers, Type: Categorical, Choices: {1, 2, 3, 4}, Default: 2
    nonlintype, Type: Categorical, Choices: {relu, tanh, sigmoid, selu}, Default: relu
  Conditions:
    hidden_size_2 | n_hidden_layers in {'2', '3', '4'}
    hidden_size_3 | n_hidden_layers in {'3', '4'}
    hidden_size_4 | n_hidden_layers in {'4'}



We can see that the MLP several hidden parameters, including categorical hyperparameters controlling the activation function and the number of hidden layers, a floating point hyperparameter controlling the learning rate, and integer hyperparameters controlling the size of each layer.

We also have conditional hyperparameter relationships.  For example, the `hidden_size_4` hyperparameter is only active when there are 4 or more hidden layers.

AutoMPC uses the [ConfigSpace](https://automl.github.io/ConfigSpace/master/) package for configurations and configuration spaces.  See the package [documentation](https://automl.github.io/ConfigSpace/master/) for more information.


From the configuration space, we can create a configuration and set it's hyperparameters as follows

The model has a current configuration as well, which is set to the defaults.

In [4]:
print(model.get_config())

Configuration(values={
  'batchnorm': False,
  'hidden_size_1': 128,
  'hidden_size_2': 128,
  'lr': 0.001,
  'n_hidden_layers': '2',
  'nonlintype': 'relu',
})



This works in a similar way for optimizers and cost transformers.

In [5]:
from autompc.ocp import QuadCostTransformer
cost = QuadCostTransformer(system)
print(cost.get_config_space())
print(cost.get_config())

Configuration space object:
  Hyperparameters:
    dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale

Configuration(values={
  'dx_F': 1.0,
  'dx_Q': 1.0,
  'omega_F': 1.0,
  'omega_Q': 1.0,
  'theta_F': 1.0,
  'theta_Q': 1.0,
  'u_R': 1.0,
  'x_F': 1.0,
  'x_Q': 1.0,
})



In [6]:
from autompc.optim import IterativeLQR
optimizer = IterativeLQR(system)
print(optimizer.get_config_space())
print(optimizer.get_config())

Configuration space object:
  Hyperparameters:
    frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    max_iter, Type: UniformInteger, Range: [10, 50], Default: 20

Configuration(values={
  'frequency': 1,
  'horizon': 20,
  'max_iter': 20,
})



To modify the current configuration of a tunable, you can then use `get_config/set_config` and treat the configuration like a dictionary. You may also use the `set_hyper_values` method.  Here we set the hyperparameters of the MLP model:

In [7]:
model_cfg = model.get_config()
model_cfg["n_hidden_layers"] = "2"
model_cfg["hidden_size_1"] = 32
model_cfg["hidden_size_2"] = 32
model.set_config(model_cfg)

model.set_hyper_values(hidden_size_1=64)

## Operations on Tunables

After a Tunable has been configured, then you can proceed to perform its configured operations.  These operations depend on the type.  For example, you can train the model using its current configuration.

In [8]:
model.train(trajs)

100%|██████████| 200/200 [00:11<00:00, 16.93it/s]


If you would like to create a slightly modified model, you can safely copy the object and reconfigure it, then re-train. We see the new model has a lower RMSE, which makes sense because it has 3 hidden layers rather than 2.

In [9]:
import copy
from autompc.sysid.metrics import get_model_rmse

model2 = copy.deepcopy(model)
model2.set_hyper_values(n_hidden_layers="3")
model2.train(trajs)
print("Training RMSE of MLP config 1:",get_model_rmse(model,trajs))
print("Training RMSE of MLP config 2:",get_model_rmse(model2,trajs))


100%|██████████| 200/200 [00:12<00:00, 16.30it/s]
Training RMSE of MLP config 1: 0.07110542035052599
Training RMSE of MLP config 2: 0.043067499801221945


Typical operations are described as follows:

Model:
- `train`
- `set_train_budget`
- `pred, pred_batch, pred_diff`

Optimizer:
- `step`
- `reset`
- `get_traj`, `get_state`, `set_state` (after `step`)

OCPTransformer:
- `__call__`

## Fixing options

When working with a tuner, you may wish to prevent the tuner from exploring some hyperparameters, and instead force the tuner to keep those values in a range or even constant, e.g., based on your intuition or results from a prior tuning run.  It is not enough to set the current configuration of the Tunable, because the tuner will overwrite the current configuration multiple times during the tuning process. 

Instead, you can use the `set_hyperparameter_bounds` and `fix_hyperparameters` methods.  The first option changes the range of the integer parameter, and the second freezes the value of a parameter.

In [10]:
optimizer.set_hyperparameter_bounds(max_iter=(5,50))
optimizer.fix_hyperparameters(frequency=1)
print(optimizer.get_config_space())
print(optimizer.get_config())

Configuration space object:
  Hyperparameters:
    frequency, Type: UniformInteger, Range: [1, 1], Default: 1
    horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    max_iter, Type: UniformInteger, Range: [5, 50], Default: 20

Configuration(values={
  'frequency': 1,
  'horizon': 20,
  'max_iter': 20,
})



## Pipelines

We use the TunablePipeline class that generate the combined configuration space for a combination of tunable components, each of which can have multiple options.  For example, a Controller consists of a model, cost transformer(s), and optimizer. 

To use a TunablePipeline, you will need to add component options to it.  The `AutoSelectController` class automatically fills the available options for you... which provides a huge list!

In [11]:
pipeline = ampc.AutoSelectController(system)
pipeline.get_config_space()

Forbidding model MLP to be used with LQR due to property is_linear
Forbidding model SINDy to be used with LQR due to property is_linear


Configuration space object:
  Hyperparameters:
    ARX:history, Type: UniformInteger, Range: [1, 10], Default: 4
    DirectTranscription:horizon, Type: UniformInteger, Range: [1, 30], Default: 10
    GaussRegTransformer:reg_weight, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    IterativeLQR:frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    IterativeLQR:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    IterativeLQR:max_iter, Type: UniformInteger, Range: [10, 50], Default: 20
    Koopman:lasso_alpha, Type: UniformFloat, Range: [1e-10, 100.0], Default: 1.0, on log-scale
    Koopman:method, Type: Categorical, Choices: {lstsq, lasso, stable}, Default: lstsq
    Koopman:poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    Koopman:poly_cross_terms, Type: Categorical, Choices: {false}, Default: false
    Koopman:poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    Koopman:trig_basis, Type: Categorical, Choi

Unlike a Tunable, a pipeline needs to be manually configured or tuned before it is used.

Compatibility between components is enforced by a variety of constraints that can be imposed on the configuration space. As an example, since the OCP here has no bounds and is quadratic, there is no need for a QuadCostFactory or bounds transformer to be applied.  Internally, in `pipeline.set_ocp()`, the constraints will be configured appropriately.  Notice the difference between this configuration space and the prior printout, that `constraint_transformer` is now `_` indicating no constraint transformer, and there is a constraint forbidding `cost_transformer` to be a QuadCostFactory.

In [12]:
from autompc import OCP
from autompc.costs import QuadCost

ocp = OCP(system)
ocp.set_cost(QuadCost(system,np.eye(4)*10,np.eye(1)*0.1,np.eye(4)*50))

pipeline.set_ocp(ocp)
pipeline.get_config_space()

Forbidding model MLP to be used with LQR due to property is_linear
Forbidding model SINDy to be used with LQR due to property is_linear
Checking compatibility with OCP
Quad cost
['Identity', 'QuadCostTransformer', 'KeepBounds', 'DeleteBounds', 'GaussRegTransformer']
Forbidding QuadCostTransformer


Configuration space object:
  Hyperparameters:
    ARX:history, Type: UniformInteger, Range: [1, 10], Default: 4
    DirectTranscription:horizon, Type: UniformInteger, Range: [1, 30], Default: 10
    GaussRegTransformer:reg_weight, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    IterativeLQR:frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    IterativeLQR:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    IterativeLQR:max_iter, Type: UniformInteger, Range: [10, 50], Default: 20
    Koopman:lasso_alpha, Type: UniformFloat, Range: [1e-10, 100.0], Default: 1.0, on log-scale
    Koopman:method, Type: Categorical, Choices: {lstsq, lasso, stable}, Default: lstsq
    Koopman:poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    Koopman:poly_cross_terms, Type: Categorical, Choices: {false}, Default: false
    Koopman:poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    Koopman:trig_basis, Type: Categorical, Choi

To specify options for each component, you can freeze them usnig `fix_option` or eliminate them using `forbid_option`.  You can also freeze the choice and the choice's hyperparameters using `set_hyper_values`.

In [13]:
pipeline.forbid_option('model','SINDy')
pipeline.get_config_space()

Forbidding model MLP to be used with LQR due to property is_linear
Forbidding model SINDy to be used with LQR due to property is_linear
Checking compatibility with OCP
Quad cost
['Identity', 'QuadCostTransformer', 'KeepBounds', 'DeleteBounds', 'GaussRegTransformer']
Forbidding QuadCostTransformer


Configuration space object:
  Hyperparameters:
    ARX:history, Type: UniformInteger, Range: [1, 10], Default: 4
    DirectTranscription:horizon, Type: UniformInteger, Range: [1, 30], Default: 10
    GaussRegTransformer:reg_weight, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    IterativeLQR:frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    IterativeLQR:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    IterativeLQR:max_iter, Type: UniformInteger, Range: [10, 50], Default: 20
    Koopman:lasso_alpha, Type: UniformFloat, Range: [1e-10, 100.0], Default: 1.0, on log-scale
    Koopman:method, Type: Categorical, Choices: {lstsq, lasso, stable}, Default: lstsq
    Koopman:poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    Koopman:poly_cross_terms, Type: Categorical, Choices: {false}, Default: false
    Koopman:poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    Koopman:trig_basis, Type: Categorical, Choi

TODO describe `forbid_incompatible_options`.